In [ ]:
import stan
import bambi as bmb
import arviz as az
import pandas as pd
import pymc as pm
import numpy as np
import statsmodels.api as sm

az.style.use("arviz-darkgrid")

In [ ]:
df = pd.read_csv("data.csv")
df

In [ ]:
with pm.Model() as model:
    # Priors for unknown model parameters
    sigma = pm.HalfCauchy("sigma", beta=10)
    intercept = pm.Normal("Intercept", 0, sigma=20)
    slope = pm.Normal("slope", 0, sigma=20)

    likelihood = pm.Normal("y", mu=intercept + slope * df["X"], sigma=sigma, observed=df["Y"])
    idata = pm.sample(3000)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_trace(idata);

In [ ]:
df.describe()

In [ ]:
# A Laplace prior with mean of 0 and scale given by a HalfNormal with a scale of 1
my_favorite_prior = bmb.Prior("Laplace", mu=0, b=bmb.Prior("HalfNormal", sigma=1))

# Set the prior when adding a term to the model; more details on this below.
priors = {"1|uid": my_favorite_prior}
bmb.Model("value ~ condition + (1|uid)", data, priors=priors, dropna=True)

In [ ]:
priors = {"X": bmb.priors}

model = bmb.Model('Y ~ X', df)

# Fit the model using 1000 on each chain
results = model.fit(draws=1000)


In [ ]:
# Key summary and diagnostic info on the model parameters
az.summary(results)

In [ ]:
# Use ArviZ to plot the results
az.plot_trace(results);